In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (5,8)
import matplotlib.font_manager
import seaborn as sns
import anndata
import scanpy as sc
import re
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")


In [3]:
data_dir = "/well/immune-rep/users/vbw431/Projects/Peppa/data/"
newdata_dir = "/well/immune-rep/users/vbw431/Projects/Peppa/new_analysis/data/"
plot_dir = "/well/immune-rep/users/vbw431/Projects/Peppa/new_out/final_plots/"


In [4]:
adata = sc.read_h5ad("/well/immune-rep/users/vbw431/Projects/Peppa/out/peppa_azi_combat.h5ad")
##load data
labels = ["NK", "CD8", "CD4", "Bcells", "Myeloid_Plt"]
cluster_list = {}
scvi_list = {}
umap_list = {}
for i in [0,1,2,3,4]:
    print("reading in " + labels[i])
    cluster_list[labels[i]] = pd.read_csv(os.path.join(newdata_dir + labels[i] +"_clustering_final/", f"Peppa_{labels[i]}_cluster_assignment.csv"), index_col =0)
    scvi_list[labels[i]] = pd.read_csv(os.path.join(newdata_dir + labels[i] +"_embeddings/", f"Peppa_{labels[i]}_scvi.csv"), index_col =0)
    umap_list[labels[i]] = pd.read_csv(os.path.join(newdata_dir + labels[i] +"_embeddings/", f"Peppa_{labels[i]}_umap.csv"), index_col =0)

adata_list = {}

for i in [0,1,2,3, 4]:
    adata_list[labels[i]] = adata[cluster_list[labels[i]].index].copy()
    adata_list[labels[i]].obsm["X_scVI"] = scvi_list[labels[i]].loc[adata_list[labels[i]].obs_names].values
    adata_list[labels[i]].obsm["X_umap"] = umap_list[labels[i]].loc[adata_list[labels[i]].obs_names].values
    adata_list[labels[i]].obs = adata_list[labels[i]].obs.merge(cluster_list[labels[i]], left_index=True, right_index=True, how="inner")
    adata_list[labels[i]].layers['counts'] = adata_list[labels[i]].X.copy()
    sc.pp.normalize_total(adata_list[labels[i]], target_sum=1e4)
    sc.pp.log1p(adata_list[labels[i]])
    adata_list[labels[i]].layers['normalized'] = adata_list[labels[i]].X.copy()
    

reading in NK
reading in CD8
reading in CD4
reading in Bcells
reading in Myeloid_Plt


In [5]:
##update meta_data with new clinical
clin_meta = pd.read_csv(newdata_dir + "index_demo.csv", index_col = 0)
clin_meta = clin_meta[["disease_group", 
                       "bio_replicate", 
                       "study_disease", 
                       "scanpy_index", 
                       "study_ID", 
                       "Treatment_status",
                      "Ethnicity",
                      "Sex",
                      "Age",
                      "HBV_serostatus",
                      "HBV_sAg_titre",
                      "HBV_DNA_VL"]]


In [6]:
new_obs = {}

for name in labels:
    del adata_list[name].obs["disease_group"]
    del adata_list[name].obs["study_disease"]
    new_df = pd.merge(adata_list[name].obs, clin_meta, how='left', left_on = ['bio_replicate','scanpy_index'], right_on = ['bio_replicate','scanpy_index'])
    new_df.index = adata_list[name].obs.index
    new_obs[name] = new_df.copy()
    adata_list[name].obs = new_df.copy()

       

In [7]:
for name in labels:
    adata_list[name].obs["celltype_consensus.l1"] = str(name)
    adata_list[name].obs["celltype_consensus.l2"] = adata_list[name].obs[str(name+".annotation.l1")]

In [8]:
## subset for only on-treatment patients as pre-treatment samples only available in some
on_treatment = {}

for name in labels:
    on_treatment[name] = adata_list[name][adata_list[name].obs["Treatment_status"] != "Pre_treatment"].copy()


In [9]:
##concat
adata_all = anndata.concat(on_treatment, join= "outer", index_unique=None)


In [10]:
##Assign CMV status based on azimuth preprint (https://doi.org/10.1016/j.cell.2021.04.048)

conditions = [
  ([re.search("_0|SeuratProject",a) is not None for a in adata_all.obs["orig.ident"]]),
  ([re.search("_0|SeuratProject",a) is None for a in adata_all.obs["orig.ident"]]),
 ]

# create a list of the values we want to assign for each condition
values = ["Subset", "Remove"]

# create a new column and use np.select to assign values to it using our lists as arguments
adata_all.obs["prevax"] = np.select(conditions, values)


In [11]:
adata_all = adata_all[adata_all.obs["prevax"] == "Subset"].copy()

## Sccoda

In [13]:
import sccoda
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz

ModuleNotFoundError: No module named 'sccoda'

In [ ]:
##Separate to on_treatment for first comparison
on_treat = adata_all[adata_all.obs["Treatment_status"] != "Pre_treatment"].copy()

In [ ]:
cov_df = on_treat.obs[['study_disease', 'bio_replicate']].drop_duplicates(subset=['study_disease', 'bio_replicate'], keep='last')
cov_df.index = cov_df["bio_replicate"]
cov_df
del cov_df["bio_replicate"]

In [ ]:
counts = pd.crosstab(on_treat.obs["bio_replicate"], on_treat.obs["celltype_consensus.l2"])
counts.columns.name = None
counts["bio_replicate"] = counts.index

In [ ]:
data_counts = dat.from_pandas(counts, covariate_columns=["bio_replicate"])
# Extract condition from and add it as an extra column to the covariates
data_counts.obs["study_disease"] = cov_df

In [ ]:
# Stacked barplot for each sample
viz.stacked_barplot(data_counts, feature_name="bio_replicate")
# Stacked barplot for the levels of "Condition"
viz.stacked_barplot(data_counts, feature_name="study_disease")


In [ ]:
prop_matrix = pd.DataFrame(data_counts.X, index = data_counts.obs_names, columns = data_counts.var_names)
prop_matrix["bio_replicate"] = prop_matrix.index
prop_meta = data_counts.obs
prop_matrix

In [ ]:
from pylab import *
cmap = cm.get_cmap('tab20', 20) 
cmap

tab20list = []
for i in range(cmap.N):
    rgba = cmap(i)
    # rgb2hex accepts rgb or rgba
    tab20list.append(matplotlib.colors.rgb2hex(rgba))
    
tab20list

In [ ]:
import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging
import rpy2.robjects.lib.ggplot2 as gp
from rpy2.robjects import pandas2ri
from rpy2.robjects import r
from rpy2.ipython.ggplot import image_png

#sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R

.libPaths(c(paste0("/well/immune-rep/users/vbw431/python/scvi_new_ivy/r_modules"), .libPaths()))

library(tidyverse)
library(RColorBrewer)

cur.dir = "/well/immune-rep/users/vbw431/Projects/Peppa/data/DIMITRA_FASTQ/"
work.dir = "/well/immune-rep/users/vbw431/Projects/Peppa/"
out.dir = "/well/immune-rep/users/vbw431/Projects/Peppa/out/"
references = "/well/immune-rep/users/vbw431/reference/reference/refdata-gex-GRCh38-2020-A/"
com.out = "/well/immune-rep/users/vbw431/Projects/Peppa/reference_combat/"

##plotting
library(ggplot2); theme_set(theme_bw(base_size = 18)+
                              theme(strip.text = element_text(colour = 'black', face="bold",size=12), 
                                    panel.grid.major = element_blank(), 
                                    panel.grid.minor = element_blank(),
                                    panel.border = element_rect(size = 0.7),
                                    axis.ticks.length=unit(.10, "cm"),
                                    axis.ticks = element_line(size=0.7),
                                    strip.background = element_blank()))



In [ ]:
%%R -i prop_matrix -i prop_meta -h 500 -w 900 -i tab20list
nb.cols <- 31
mycolors <- colorRampPalette(tab20list)(nb.cols)


prop_plot <- prop_matrix %>%
reshape2::melt(id.vars = c("bio_replicate")) %>%
left_join(prop_meta, by=c("bio_replicate")) %>%
mutate(`Cell Type` = ifelse(grepl("Prolifering", variable), "NK.Proliferating", as.character(variable))) %>%
#mutate(`Cell Type` = variable) %>%
ggplot(aes(bio_replicate, value, fill = `Cell Type`))+
geom_col(position = "fill")+
facet_wrap(~study_disease, nrow=1, scales = "free_x")+
theme(axis.text.x = element_text(angle=45, hjust=1))+
scale_fill_manual(values = mycolors)+
ylab("Proportion")+
xlab("")

print(prop_plot)

prop_matrix %>%
reshape2::melt(id.vars = c("bio_replicate")) %>%
left_join(prop_meta, by=c("bio_replicate")) %>%
mutate(`Cell Type` = variable) %>%
ggplot(aes(`Cell Type`, value, fill = study_disease))+
geom_boxplot(outlier.alpha = 0)+
#geom_point(aes(color=study_disease), position = position_jitterdodge(jitter.width=0.1))+
theme(axis.text.x = element_text(angle=45, hjust=1))+
scale_fill_manual(values = mycolors)+
scale_color_manual(values = mycolors)+
ylab("Proportion")+
xlab("")



In [ ]:
from sccoda.util import comp_ana as mod


In [ ]:
# model all three diseases at once
model_all = mod.CompositionalAnalysis(data_counts, formula="study_disease")
all_results = model_all.sample_hmc()
all_results.set_fdr(est_fdr=0.4)
all_results.summary()

In [ ]:
all_results.credible_effects()


In [ ]:
credible_effects_HBV = all_results.credible_effects()["study_disease[T.HBV]"]
credible_effects_HBV_HIV = all_results.credible_effects()["study_disease[T.HBV_HIV]"]


In [ ]:
HBV_effect = all_results.effect_df.loc["study_disease[T.HBV]"]
HBV_HIV_effect = all_results.effect_df.loc["study_disease[T.HBV_HIV]"]

HBV_effect = HBV_effect.loc[credible_effects_HBV].reset_index()
HBV_HIV_effect = HBV_HIV_effect.loc[credible_effects_HBV_HIV].reset_index()

In [ ]:
%%R -i HBV_effect -i HBV_HIV_effect -h 700 -w 1200
library(tidytext)

HBV_effect$contrast <- "HBV vs. rest"
HBV_HIV_effect$contrast <- "co-infection vs. rest"

combined <- rbind(HBV_effect, HBV_HIV_effect)

#combined %>% 
#ggplot(aes(reorder_within(`Cell Type`, `log2-fold change`, contrast), `log2-fold change`, fill = `log2-fold change`))+
#geom_col()+
#facet_wrap(~contrast, scales="free")+
#scale_x_reordered()+
#theme(axis.text.x = element_text(angle=45, hjust=1))+
# scale_fill_gradientn(colors = colorRampPalette(c("#276DAA", "white", "#DD1C28"))(100))+
#xlab("")

sccoda_prop <- combined %>% 
mutate(Contrast = contrast) %>%
mutate(`Parent Cell Type` = ifelse(grepl("^B.", `Cell Type`), "B cells",
                                  ifelse(grepl("^CD4.", `Cell Type`), "CD4",
                                        ifelse(grepl("^CD8.|MAIT|MPECs|gdT", `Cell Type`), "CD8",
                                              ifelse(grepl("^NK.", `Cell Type`), "NK", "Myeloid_Plt"))))) %>%
ggplot(aes(`Cell Type`, `log2-fold change`, color = Contrast))+
geom_point(aes(size=`Inclusion probability`), position = position_dodge(width = .8))+
geom_errorbar(aes(ymin=0, ymax = `log2-fold change`),
                   width = 0,
                   position = position_dodge(width = .8), size=0.5)+
geom_hline(yintercept = 0, linetype="dashed")+
geom_vline(lty = 2, xintercept=seq(from = 0.5, to = length(unique(combined$`Cell Type`)), by = 1),color="gray", size = 0.5)+

#facet_wrap(~contrast, scales="free")+
scale_x_reordered()+
theme(axis.text.x = element_text(angle=45, hjust=1))+
 #scale_fill_gradientn(colors = colorRampPalette(c("#276DAA", "white", "#DD1C28"))(100))+
ggsci::scale_color_d3()+
xlab("")+
facet_grid(vars(`Parent Cell Type`), scales="free_y")+
coord_flip()
print(sccoda_prop)

ggsave(sccoda_prop, file="/well/immune-rep/users/vbw431/Projects/Peppa/new_out/final_plots/sccoda_prop.eps", device="eps",  width = 9, height = 11, units = "in")

ggsave(prop_plot, file="/well/immune-rep/users/vbw431/Projects/Peppa/new_out/final_plots/prop_fill.eps", device="eps",  width = 14, height = 7, units = "in")





In [ ]:
%%R  -h 500 -w 400

##NK only

NK <- combined %>% 
filter(grepl("NK", `Cell Type`)) %>%
mutate(`Cell Type` = ifelse(grepl("Prolifering", `Cell Type`), "NK.Proliferating", `Cell Type`)) 

NK %>%
ggplot(aes(reorder(`Cell Type`, `log2-fold change`), `log2-fold change`, fill = `log2-fold change`))+
geom_col()+
facet_grid(vars(contrast))+
theme(axis.text.x = element_text(angle=45, hjust=1))+
 scale_fill_gradientn(colors = colorRampPalette(c("#276DAA", "white", "#DD1C28"))(100))+
xlab("")



In [ ]:
HBV_HIV_effect

In [ ]:
HBV_effect

In [ ]:
import altair as alt
from altair_saver import save

In [ ]:
charthbv = alt.Chart(
        HBV_effect.loc[credible_effects_HBV].reset_index(),
        title="HBV vs. Rest",
    ).mark_bar().encode(
        x=alt.X("Cell Type", sort="y"),
        y="log2-fold change",
        color=alt.Color("Cell Type"),
    )

charthbv

In [ ]:
charthbvhiv = (
    alt.Chart(
        HBV_HIV_effect.loc[credible_effects_HBV_HIV].reset_index(),
        title="HBV_HIV vs. Rest",
    )
    .mark_bar()
    .encode(
        x=alt.X("Cell Type", sort="y"),
        y="log2-fold change",
        color=alt.Color("Cell Type"),
    )
)
charthbvhiv

In [ ]:

##plot on umap

on_treat.obs["HBV_effect_sscoda"] = [
    all_results.effect_df.loc[("study_disease[T.HBV]", c), "log2-fold change"]
    for c in on_treat.obs["celltype_consensus.l2"]
]
on_treat.obs["HBV_HIV_effect_sscoda"] = [
    all_results.effect_df.loc[("study_disease[T.HBV_HIV]", c), "log2-fold change"]
    for c in on_treat.obs["celltype_consensus.l2"]
]



In [ ]:
for i in labels:
    sc.pl.umap(
        on_treat[on_treat.obs["celltype_consensus.l1"] == str(i)],
        color=[
            "celltype_consensus.l2",
            "HBV_effect_sscoda",
            "HBV_HIV_effect_sscoda",
        ],
        ncols=3,
        wspace=0.25,
        vcenter=0,
        vmax=1.5,
        vmin=-1.5,
        cmap = 'RdBu_r',
        add_outline = True,
        size=10
    )
    plt.show()

## Repeat sccoda to compare pre and post treatment HBV

In [ ]:
paired_tre = adata_all[adata_all.obs["Treatment_status"].isin(["Pre_treatment", "On_treatment"])].copy()
set(paired_tre.obs["bio_replicate"])
paired_tre = paired_tre[paired_tre.obs["bio_replicate"].isin(["patient_1", 
                                                              "patient_2",
                                                              "patient_3",
                                                              "patient_6",
                                                              "patient_7"])].copy()

In [ ]:
paired_tre.obs["treat_rep"] = paired_tre.obs['Treatment_status'].astype(str) +paired_tre.obs['bio_replicate'].astype(str)



In [ ]:
set(paired_tre.obs["treat_rep"])

In [ ]:
cov_tr = paired_tre.obs[['Treatment_status', 'treat_rep']].drop_duplicates(subset=['Treatment_status', 'treat_rep'], keep='last')

cov_tr.index = cov_tr["treat_rep"]
cov_tr
del cov_tr["treat_rep"]


In [ ]:
paired_tre.obs["treat_rep"]

In [ ]:
counts_tr = pd.crosstab(paired_tre.obs["treat_rep"], paired_tre.obs["celltype_consensus.l2"])
counts_tr.columns.name = None
counts_tr["treat_rep"] = counts_tr.index

In [ ]:
data_counts_tr = dat.from_pandas(counts_tr, covariate_columns=["treat_rep"])


In [ ]:
# Extract condition and add it as an extra column to the covariates
data_counts_tr.obs["Treatment_status"] = cov_tr
print(data_counts_tr.X)
print(data_counts_tr.obs)

In [ ]:
# Grouped boxplots. No facets, relative abundance, no dots.
viz.boxplots(
    data_counts_tr,
    feature_name="Treatment_status",
    plot_facets=False,
    y_scale="relative",
    add_dots=False,
)
plt.savefig(plot_dir + "sscoda_prop_boxplot_prevspost.svg",)


In [ ]:
# model all three diseases at once
model_all_tr = mod.CompositionalAnalysis(data_counts_tr, formula="Treatment_status")
all_results_tr = model_all_tr.sample_hmc()
all_results_tr.summary()

In [ ]:
all_results_tr.set_fdr(est_fdr=0.4)
all_results_tr.credible_effects()


In [ ]:
## No changes in composition pre and post treatment on sscoda